In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime as dt
import requests
import pandas as pd

In [3]:
url = ("https://financialmodelingprep.com/api/v3/ipo_calendar?from=2020-01-01&to=2020-09-01&apikey=demo")

root = f'https://financialmodelingprep.com/api/v3/'
FMP_API_KEY = '3872e65004997436bb78a980862dc1ac'
endpoint = 'ipo_calendar'
suffix = f'apikey={FMP_API_KEY}'

In [4]:
rng = pd.date_range(end=dt.now(), periods=89, freq='D')

In [5]:
start = rng[0]
end = rng[-1]

In [6]:
param_rngs = []
for i in range(4):
    params = {
        'from': start.strftime('%Y-%m-%d'),
        'to': end.strftime('%Y-%m-%d')
    }
    param_rngs.append(params)
    rng = pd.date_range(end=start - pd.Timedelta(days=1), periods=89, freq='D')
    start = rng[0]
    end = rng[-1]

In [7]:
dfs = []
for params in param_rngs:
    query_params = '?'
    for k, v in params.items():
        query_params += f'{k}={v}&'

    query_params += suffix
    url = f'{root}{endpoint}{query_params}'
    res = requests.get(url)
    df = pd.DataFrame(res.json())
    dfs.append(df)

In [8]:
ipos_fmp = pd.concat(dfs)

In [9]:
na_mkts = ['Nasdaq', 'NYSE', 'NYSE American', 'TSXV', 'Canadian Sec', 'Toronto']

In [10]:
ipos_fmp_na = ipos_fmp[ipos_fmp.exchange.isin(na_mkts)]

In [34]:
ipos_fmp_na

,date,company,symbol,exchange,actions,shares,priceRange,marketCap
0,2021-07-09,Belong Acquisition Corp.,BLNGU,Nasdaq,Expected,NaN,10.00 - 10.00,NaN
1,2021-07-09,Sentage Holdings Inc. Ordinary Shares,SNTG,Nasdaq,Expected,NaN,5.00 - 6.00,NaN
2,2021-07-09,Galata Acquisition Corp.,GLTAU,NYSE American,Expected,NaN,10.00 - 10.00,NaN
3,2021-07-09,Skydeck Acquisition Corp. Class A Ordinary Shares,SKYA,Nasdaq,Expected,NaN,None,NaN
5,2021-07-09,"TransCode Therapeutics, Inc. Common Stock",RNAZ,Nasdaq,Expected,NaN,4.00 - 4.00,NaN
...,...,...,...,...,...,...,...,...
269,2020-08-06,"Rocket Companies, Inc.",RKT,NYSE,priced,100000000.0,18.0,1.800000e+09
274,2020-08-05,GO Acquisition Corp.,GOACU,NYSE,priced,50000000.0,10.0,5.000000e+08
276,2020-08-04,Yucaipa Acquisition Corp,YAC'U,NYSE,priced,30000000.0,10.0,3.000000e+08
287,2020-07-23,"Montrose Environmental Group, Inc.",MEG,NYSE,priced,10000000.0,15.0,1.500000e+08


In [11]:
ipos_fmp_na[ipos_fmp_na.symbol.str.contains('WAVE')]

,date,company,symbol,exchange,actions,shares,priceRange,marketCap
77,2021-07-01,Eco Wave Power Global AB (publ) American Depos...,WAVE,Nasdaq,Expected,NaN,None,NaN


In [12]:
ipos_fmp_na.reset_index(drop=True).sort_values(by='marketCap', ascending=False)

,date,company,symbol,exchange,actions,shares,priceRange,marketCap
1838,2020-07-22,"Pershing Square Tontine Holdings, Ltd.",PSTHU,NYSE,priced,200000000.0,20.00,4.000000e+09
1753,2020-12-09,DoorDash Inc,DASH,NYSE,Priced,33000000.0,102.00,3.366000e+09
1815,2020-09-16,Snowflake Inc.,SNOW,NYSE,priced,28000000.0,120.00,3.360000e+09
1829,2020-08-13,KE Holdings Inc.,BEKE,NYSE,priced,106000000.0,20.00,2.120000e+09
1806,2020-09-30,Palantir Technologies Inc.,PLTR,NYSE,priced,257135415.0,7.25,1.864232e+09
...,...,...,...,...,...,...,...,...
1777,2020-10-28,MAX,MAX,NYSE,Expected,NaN,19.00,NaN
1778,2020-10-28,BOACU,BOACU,NYSE,Expected,NaN,10.00,NaN
1779,2020-10-28,ACICU,ACICU,NYSE,Expected,NaN,10.00,NaN
1780,2020-10-28,ROOT,ROOT,Nasdaq,Expected,NaN,22.00,NaN


In [20]:
url = 'https://www.iposcoop.com/last-12-months/'
ipos = pd.read_html(url)[0]
ipos = ipos.rename(columns={
    'Shares (millions)': 'Shares',
    'Offer Price': 'Offer'
})
ipos.iloc[:, 5:8] = ipos.iloc[:, 5:8].apply(lambda col: col.str.replace('$', '', regex=False).astype(float))

In [21]:
ipos['Raise'] = ipos.Shares * ipos.Offer

In [26]:
industries = [
#     'Technology', 
#     'Industrials', 
#     'Other',
#     'Blank Check'
#     'Consumer Services'm,
    'Technology',
]

In [27]:
ipos_tech = ipos[ipos.Industry.isin(industries)].sort_values(by='Raise', ascending=False)

In [28]:
ipos_tech.head(40)

,Company,Symbol,Industry,Offer Date,Shares,Offer,1st Day Close,Current Price,Return,SCOOP Rating,Raise
176,Roblox Corporation,RBLX,Technology,3/10/2021,198.9,45.00,69.88,86.26,91.69%,S/O,8950.500
15,DiDi Global Inc.,DIDI,Technology,6/30/2021,317.0,14.00,14.14,12.03,-14.07%,S/O,4438.000
350,Snowflake,SNOW,Technology,9/16/2020,28.0,120.00,253.93,269.49,124.58%,S/O,3360.000
197,Bumble Inc.,BMBL,Technology,2/11/2021,50.0,43.00,70.31,53.30,23.95%,S/O,2150.000
81,"Squarespace, Inc.",SQSP,Technology,5/19/2021,40.4,50.00,43.65,56.00,12.00%,S/O,2020.000
119,AppLovin Corporation,APP,Technology,4/15/2021,25.0,80.00,65.20,66.18,-17.28%,S/O,2000.000
231,Shoals Technologies Group,SHLS,Technology,1/27/2021,77.0,25.00,30.98,35.38,41.52%,S/O,1925.000
242,Playtika Holding Corp.,PLTK,Technology,1/15/2021,69.5,27.00,31.62,23.10,-14.44%,S/O,1876.500
327,Palantir Technologies,PLTR,Technology,9/30/2020,257.1,7.25,9.50,23.29,221.24%,S/O,1863.975
77,ZipRecruiter,ZIP,Technology,5/26/2021,86.6,20.00,21.10,24.32,21.60%,S/O,1732.000


In [30]:
# ipos_tech.Raise.sum()
ipos.shape

(393, 11)

In [35]:
ipos[ipos.Raise < 100].shape

(103, 11)

+ ARRY = solar trackers
+ CRCT = at-home fabric cutting machines and consumables
+ KRT = eco-friendly packaging products
+ CNEY = wood-based activated carbon used in pharma and industrial manufacturing
+ FIGS = manufacturer customizable medical scrubs
+ EZGO = design and manufacturer of e-bikes
+ VIAO = digital display products for automotive (CHIP and CIRCUIT BOARD MAKERS TOO!)
+ UTME = manufacturer and design of low-cost phones and components

SPACs and investment funds

https://www.ft.com/content/d65f539f-8842-4640-89b0-8de9df9efcc9

In [36]:
ipos[ipos.Company.str.contains('Hydrofarm')]

,Company,Symbol,Industry,Offer Date,Shares,Offer,1st Day Close,Current Price,Return,SCOOP Rating,Raise
268,"Hydrofarm Holdings Group, Inc.",HYFM,Consumer Goods,12/10/2020,8.7,20.0,51.99,57.74,188.70%,S/O,174.0


SPAC Deals:

Lightning eMotors
https://lightningemotors.com/lightning-emotors-and-gigcapital3-announce-closing-of-business-combination/
Joby Aviation
https://a.reinventtechnologypartners.com/investor-relations/press-releases/detail/6/joby-aviation-to-list-on-nyse-through-merger-with-reinvent

Green Funding Rounds
https://www.esgtoday.com/green-steel-startup-h2gs-raises-105-million-in-equity-financing/

In [37]:
from typing import Union, Iterable
import operator as op
import numpy as np

@pd.api.extensions.register_dataframe_accessor('search')
class PandasSearch:
    """
    More user-friendly interface for searching columns and rows
    """

    CUSTOM_OPERATORS = ['notin', 'isin', 'contains', 'not_contain']
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def _isin(self, ser:pd.Series, term:Iterable):
        return ser.isin(term)        
        
    def _notin(self, ser:pd.Series, term:Iterable):
        return ~ser.isin(term)
    
    def _not_contain(self, ser:pd.Series, term:[str, Iterable]):
        if isinstance(term, (list, tuple, set, np.ndarray)):
            mask = ser.str.contains(term[0])
            for ter in term[1:]:
                mask |= ser.str.contains(ter)
        elif isinstance(term, str):
            mask = ser.str.contains(term)
        return ~mask
        
    def _single_mask(self, 
        series:str, 
        operator:[str], 
        term:[str, float, int], 
        axis=0):
        """
        """
        if operator in self.CUSTOM_OPERATORS:
            operator = getattr(self, f'_{operator}')
            mask = operator(self._obj[series], term)
        else:
            operator = getattr(op, operator) if isinstance(operator, str) else operator
            mask = operator(self._obj[series], term)
        return mask
    
    def _single_search(self, *args, **kwargs):
        mask = self._single_mask(*args, **kwargs)
        return self._obj[mask]
    
    def _multi_search(self, searches):
        mask = self._single_mask(*searches[0])
        for args in searches[1:]:
            mask &= self._single_mask(*args)
        return self._obj[mask]
    
    def search(self, *args, **kwargs):
        if (len(args) > 1 or len(kwargs) > 1):
            ret = self._single_search(*args, **kwargs)
        elif isinstance(args[0], (list, tuple, dict)):
            if len(args[0]) == 1:
                ret = self._single_search(*args[0])
            else:
                ret = self._multi_search(args[0])                
        return ret

In [42]:
closed_url = "https://sheet2site-staging.herokuapp.com/api/v3/index.php?key=1ataJJQSe-DMwHk5QY7vgjWB-YkXy_aFNEBnO24Juvm8&amp;g=1&amp;e=1&amp;g=1&amp;e=1"
closed_spacs = pd.read_html(closed_url)[0]
new_names = [
    'symbol', 
    'name', 
    'spac_name', 
    'completion_date', 
    'mkt_cap', 
    'tgt_industry',
    'underwriter',
    'sector_tags'
]

closed_spacs = closed_spacs[closed_spacs.columns[[0,1,3,4,10,15,17,-2]]]
renames = {old: new for old, new in zip(closed_spacs.columns, new_names)}
closed_spacs = closed_spacs.rename(columns=renames)

closed_spacs.mkt_cap = closed_spacs.mkt_cap.str.replace('$', '', regex=False).str.replace(',','',regex=False).astype(float)

In [43]:
exclude_ind = [
    'Healthcare', 
    'Cannabis', 
    'Fintech', 
    'PropTech', 
    'Proptech',
    'InsureTech',
    'Senior',
    'China',
    'India',
    'Mexico'
]
searches = [
    ['mkt_cap', 'lt', 2.5e9],
    ['tgt_industry', 'not_contain', exclude_ind]
]
sois = closed_spacs.search.search(searches).sort_values(by='mkt_cap', ascending=False)

In [44]:
closed_spacs.head(2)

,symbol,name,spac_name,completion_date,mkt_cap,tgt_industry,underwriter,sector_tags
0,SUNL,Sunlight Financial Holdings Inc.,Spartan Acquisition Corp. II (SPRQ),2021-07-09,NaN,Energy,Cantor,"Cloud, HR Software, SaaS, Tech"
1,FREY,FREYR Battery,Alussa Energy Acquisition Corp (ALUS),2021-07-09,316250006.0,Oil and Gas,BTIG,"Cloud, HR Software, SaaS, Tech"


In [207]:
closed_spacs.search.search('symbol', 'isin', comps)

,symbol,name,spac_name,completion_date,mkt_cap,tgt_industry,underwriter,sector_tags
1,FREY,FREYR Battery,Alussa Energy Acquisition Corp (ALUS),2021-07-09,3.162500e+08,Oil and Gas,BTIG,"Cloud, HR Software, SaaS, Tech"
13,ELMS,"Electric Last Mile, Inc.",Forum Merger III Corporation (FIII),2021-06-25,1.205542e+09,NaN,Jefferies,NaN
30,JBI,"Janus International Group, Inc.",Juniper Industrial Holdings Inc (JIH),2021-06-07,1.800932e+09,Industrials,UBS,NaN
38,ZEV,"Lightning eMotors, Inc.",GigCapital3 Inc (GIK),2021-05-06,5.580102e+08,"TMT (Tech, Media, Telecom)","Nomura, Oppenheimer",NaN
43,ASTS,AST SpaceMobile,New Providence Acquisition Corp (NPA),2021-04-06,2.316318e+09,Consumer Goods,"BTIG, I-Bankers",NaN
50,PCT,"PureCycle Technologies, Inc.",Roth CH Acquisition I Co (ROCH),2021-03-17,2.372801e+09,"Business Services, Consumer, Healthcare, Tech,...","Roth Capital Partners, Craig-Hallum Capital Group",NaN
51,AEVA,"Aeva, Inc.",Interprivate Acquisition Corp (IPV),2021-03-12,2.291681e+09,NaN,"Morgan Stanley, EarlyBirdCapital, Inc.",NaN
52,OUST,"Ouster, Inc.",Colonnade Acquisition Corp. (CLA),2021-03-11,1.990975e+09,"Natural Resources, Energy, Real Estate, Agricu...",BTIG,NaN
53,VIEW,"View, Inc.",CF Finance Acquisition Corp II (CFII),2021-03-08,1.664431e+09,"Financial Services, Health-care, Real Estate S...",Cantor,NaN
61,ADN,"Advent Technologies Holdings, Inc.",AMCI Acquisition Corp (AMCI),2021-02-04,4.087006e+08,"Natural Resources, Mining","Jefferies, UBS",NaN


In [210]:
pending_url = 'https://stockmarketmba.com/pendingspacmergers.php'
pending_spacs = pd.read_html(pending_url)[0]
pending_spacs.shape

(96, 12)

In [50]:
import os
from pathlib import Path
from datetime import date, datetime as dt
import numpy as np
import pandas as pd
import scipy.stats as scist
import requests


from bs4 import BeautifulSoup as BS
from tqdm.auto import tqdm

import yfinance as yf

class Stock:
    ENDPOINTS = [
        ('profile', None),
        ('income-statement', None),
        ('income-statement', {'period': 'quarter'}),
        ('balance-sheet-statement', None), 
        ('balance-sheet-statement', {'period': 'quarter'}),
        ('cash-flow-statement', None),
        ('cash-flow-statement', {'period': 'quarter'}),
        ('ratios', None),
        ('ratios', {'period': 'quarter'}),
        ('ratios-ttm', None),
        ('key-metrics', None),
        ('key-metrics', {'period': 'quarter'}),
        ('key-metrics-ttm', None),
    ]

    def __init__(self, symbol:str, fin_params:dict={}, fmp_api_key:str='', update:bool=False,
        update_opts:bool=True, leave_pbar:bool=True
    ):
        self.symbol = symbol

        if '.TO' in symbol:
            self.ticker = CADTicker(symbol)
        else:
            self.ticker = yf.Ticker(symbol)

        self.update = update
        self.update_opts = update_opts

        self.fmp_api_key = fmp_api_key
        self.pkl_path = Path(os.path.join(os.path.abspath(''), 'pickles', symbol))

        endpoints = fin_params if fin_params else self.ENDPOINTS
        self.get_financials(*endpoints)
        self.leave_pbar = leave_pbar
#         self._fetch_options()

    def __repr__(self):
        return self.symbol

    @property
    def price(self):
        if not hasattr(self, '_price'):
            price = np.mean([self.ticker.info['bid'], self.ticker.info['ask']])
            if not price:
                price = self.ticker.history().iloc[-1].loc['Close']
            self._price = price
        return self._price

    @property
    def istat(self):
        return self.income_statement

    @property
    def bs(self):
        return self.balance_sheet_statement

    @property
    def cfstat(self):
        return self.cash_flow_statement

    def is_pickled(self, endpoint, period:str=''):
        if not self.pkl_path.exists():
            self.pkl_path.mkdir()
            return False
        else:
            if (self.pkl_path / f'{endpoint}{period}.pkl').exists():
                return True
            else:
                return False

    def call_fmp(self, endpoint, query):
        try:
            FMP_API_KEY = '3872e65004997436bb78a980862dc1ac'
            fmp_api_key = self.fmp_api_key if self.fmp_api_key else FMP_API_KEY
        except NameError as e:
            if  "name 'FMP_API_KEY' is not defined" in repr(e):
                fmpurl = 'https://financialmodelingprep.com/developer/docs/'
                msg = 'An API key is required to download financial info.'
                msg += ' The key can either be passed as a parameter to `fmi_api_key` '
                msg += ' or it can be stored as `FMP_API_KEY` in a .env at the top of the package.'
                msg += f' More details: {fmpurl}'
                raise ValueError(msg)

        root = f'https://financialmodelingprep.com/api/v3/'
        query_params = '?'
        if query is not None:
            for param, val in query.items():
                query_params += f'{param}={val}&'

        query_params += f'limit=5&apikey={fmp_api_key}'
        
        url = f'{root}{endpoint}/{self.symbol}{query_params}'
        res = requests.get(url)
        rjson = res.json()
        df = pd.DataFrame(res.json())
        df = df.T
        if not df.empty:
            if endpoint == 'profile':
                df = df.rename(columns=df.iloc[0]).drop([df.index[0]])
            else:
                if 'date' in df.columns and 'symbol' in df.columns:
                    l0 = df.loc['date']
                    l1 = df.loc['symbol']
                    df = df.T.set_index(['symbol', 'date'], drop=True)

                df = df.T
        return df

    def get_financials(self, *endpoints):
        """
        Pulls in financial data and assigns to various attributes

        API via https://financialmodelingprep.com/developer/docs/
        
        Params
        -------
            endpoints; provide endpoint: True with endpoint derived from API
        """
        for endpoint, query in endpoints:
            period = f'-{query["period"]}' if query is not None else ''
            if self.is_pickled(endpoint, period) and not self.update:
                df = pd.read_pickle(self.pkl_path / f'{endpoint}{period}.pkl')
            else:
                df = self.call_fmp(endpoint, query)
                df.to_pickle(self.pkl_path / f'{endpoint}{period}.pkl')

            attrname = f'{endpoint}{period}'.replace('-', '_')
            setattr(self, attrname, df)

    def _fetch_options(self):
        from analysis.options.assets import Put, Call

        self.calls = []
        self.puts = []        
        opt_types = [self.calls, self.puts]
        labels = ['calls', 'puts']
        yields = {}
        now = dt.now()
        min_ttm = 3

        pbar_outer = tqdm(labels, leave=self.leave_pbar)
        for label in pbar_outer:
            endpoint = f'{label}-{now.strftime("%Y-%m-%d %H")}'
            if self.is_pickled(endpoint):
                setattr(self, label, pd.read_pickle(self.pkl_path / f'{endpoint}.pkl'))
            elif not self.update_opts:
                opt_files = list(self.pkl_path.glob(f'calls-*.pkl'))
                by_date = [
                    dt.strptime(f.name.split('calls-')[-1].split('.pkl')[0], '%Y-%m-%d %H') 
                    for f in opt_files
                ]
                argmax = np.array(by_date, dtype=np.datetime64).argmax()
                setattr(self, label, pd.read_pickle(opt_files[argmax]))
            else:
                pbar_outer.set_description('No pickle. Fetching and merging.')
                for exp in self.ticker.options:
                    chains = self.ticker.option_chain(exp)
                    opts_by_exp = getattr(chains, label)
                    if not opts_by_exp.empty:
                        opts_by_exp.loc[:, 'expiry'] = exp
                        getattr(self, label).append(opts_by_exp)

                df = pd.concat(getattr(self, label))
                df = df.sort_values(by=['expiry', 'strike'])
                setattr(self, label, pd.concat(getattr(self, label)))
        
                pbar_outer.set_description('Analyzing.')
                
                opts = getattr(self, label)
                
                opts = opts.reset_index(drop=True)
                
                new_cols = [
                    'T', 't', 'r', 'r_series', 'P_calcd', 'ivol_calcd', 'P_from_greeks',
                    'delta_comp', 'gamma_comp', 'theta_comp',
                    'delta', 'gamma', 'theta', 'vega'
                ]
                opts[new_cols] = np.nan

                opts.loc[:, 'bas'] = opts[['bid', 'ask']].mean(axis=1)
                opts.loc[:, 'marketPrice'] = np.where(opts.bas == 0, opts.lastPrice, opts.bas)

                pbar_inner = tqdm(total=opts.shape[0], leave=False)
                for i, opt in opts.iterrows():
                    T = dt.strptime(opt.expiry, '%Y-%m-%d') - dt.now()
                    t = T.days / 365

                    opts.loc[i, 'T'] = T.days
                    opts.loc[i, 't'] = t
                    if T.days > min_ttm:
                        pbar_inner.set_description(f'{opt.expiry} ${opt.strike}')

                        P = opt.marketPrice
                        K = opt.strike

                        ivol_repd = opt.impliedVolatility
                    
                        i_yield = np.abs(yield_days[1:] - T.days).argmin() + 1
                        yield_label = yield_series[i_yield]
                        if yield_label not in yields:
                            yields[yield_label] = get_fred([yield_label]).iloc[-1, 0]
                        r = yields[yield_label]
                        
                        opts.loc[i, 'r'] = r
                        opts.loc[i, 'r_series'] = yield_label

                        if label == 'calls':
                            opt_obj = Call(self, **opts.loc[i].to_dict())
                        elif label == 'puts':
                            opt_obj = Put(self, **opts.loc[i].to_dict())
                        
                        opts.loc[i, 'P_calcd'] = opt_obj.find_price()
                        opts.loc[i, 'P_from_greeks'] = opt_obj.price_from_greeks()
                        opts.loc[i, ['delta_comp', 'gamma_comp', 'theta_comp']] = opt_obj.price_by_components()
                        opts.loc[i, ['delta', 'gamma', 'theta', 'vega']] = opt_obj.greeks()
                        try:
                            opts.loc[i, 'ivol_calcd'] = opt_obj.find_ivol(maxiter=10000)
                        except RuntimeError as e:
                            opts.loc[i, 'ivol_calcd'] = np.nan

                        opts.loc[i, 'opt_obj'] = opt_obj

                    pbar_inner.update(1)
                opts.expiry = pd.to_datetime(opts.expiry)
                opts.loc[:, 'p_ovc'] = opts.P_calcd - opts.marketPrice
                opts.loc[:, 'ivol_ovc'] = opts.ivol_calcd - opts.impliedVolatility
                opts.loc[:, 'theta_per'] = opts.theta / 365
                opts.loc[:, 'vega_per'] = opts.vega * .01
                opts = opts[opts.loc[:,'T'] > min_ttm].reset_index(drop=True)

                opts.to_pickle(self.pkl_path / f'{endpoint}.pkl')
        
                setattr(self, label, opts)

    def daily_vol(self, period='max', n_days=365):
        praxn = self.ticker.history(period=period)
        log_delta = np.log(praxn.Close.shift(1) / praxn.Close)
        log_delta = log_delta.iloc[1:]

        deltadist = scist.norm(*scist.norm.fit(log_delta))
        hvol = deltadist.std()*np.sqrt(n_days)

        return hvol


In [51]:
class Stocks(np.ndarray):
    def __new__(cls, stocks, info=None):
        if isinstance(stocks, dict):
            tups = [(sym, s) for sym, s in stocks.items()]
            types = [('sym', 'U10'), ('stock', Stock)]
            stocks = np.array(tups, dtype=types)
            
        obj = np.asarray(stocks).view(cls)

        return obj

    def __array_finalize__(self, obj):
        if obj is None: return
#         self.info = getattr(obj, 'info', None)

    def search(self, sym):
        if isinstance(sym, str):
            idx = np.argwhere(self['sym'] == sym).flatten()[0]
            return self[idx]['stock']
        elif isinstance(sym, (list,tuple,set,np.ndarray)):
            mask = False
            for stock in sym:
                mask |= self['sym'] == stock
            return self[np.argwhere(mask).flatten()]
    
    @property
    def has_info(self):
        has_info = {}
        for sym, stock in self:
            if not stock.ratios.isna().all().all():
                has_info[sym] = stock
        return Stocks(has_info)
    
    def comps(self, endpoint, cols=None):
        if cols is None:
            if endpoint == 'profile':
                cols = ['companyName', 'exchangeShortName', 'description', 'mktCap']
                return pd.concat([getattr(s, endpoint).T.loc[:, cols] for s in self['stock'] if not getattr(s, endpoint).empty])
            else:
                return pd.concat([getattr(s, endpoint).loc[:, getattr(s, endpoint).columns] for s in self['stock'] if not getattr(s, endpoint).empty])

        

In [52]:
comps = [
    'FREY',
    'PCT',
    'ASTS',
    'AEVA',
    'GOEV',
    'DNMR',
    'OUST',
    'JBI',
    'VIEW',
    'ELMS',
    'XL',
    'RMO',
    'EOSE',
    'ZEV',
    'ASLE',
    'ADN',
    'HTOO'
] + ['ARRY', 'CRCT', 'KRT', 'FIGS', 'EZGO', 'VIAO']

In [54]:
stocks = {comp: Stock(comp) for comp in comps}
stocks = Stocks(stocks)

In [62]:
stocks.search(comps).comps('istat').search.search('date', 'eq', '2020-12-31').T

,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
date,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31
symbol,ASTS,AEVA,GOEV,DNMR,OUST,VIEW,XL,RMO,EOSE,ASLE,ADN,ARRY,CRCT,KRT,VIAO
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,EUR
fillingDate,2021-05-06,2021-06-02,2021-03-31,2021-03-30,2021-03-09,2021-02-16,2021-05-17,2021-04-15,2021-02-26,2021-03-16,2021-03-26,2021-03-10,2020-12-31,2020-12-31,2020-12-31
acceptedDate,2021-05-06,2021-06-02,2021-03-31,2021-03-30,2021-03-09,2021-02-16,2021-05-17,2021-04-15,2021-02-26,2021-03-16,2021-03-26,2021-03-09 19:46:13,2020-12-31,2020-12-31,2020-12-31
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenue,5967000.000000,0.000000,2550.000000,47333000,0.000000,32302000,20338000.000000,8974000.000000,219000.000000,208938247,0.000000,872662000,959030000,295518000,152600000
costOfRevenue,3025000.000000,2741000.000000,670.000000,35876000,0.000000,123110000,17594000.000000,17965000.000000,5509000.000000,230282599,0.000000,669861000,626656000,206393000,129300000
grossProfit,2942000.000000,2102000.000000,1880.000000,11457000,0.000000,-90808000,2744000.000000,-8991000.000000,-5290000.000000,52789145,0.000000,202801000,332374000,89125000,23300000
grossProfitRatio,0.493045,0.000000,0.737255,0.242051,0.000000,-2.811219,0.134920,-1.001894,-24.155251,0.252654,0.000000,0.232394,0.346573,0.301589,0.152687


In [66]:
stocks.search(comps).comps('key_metrics').search.search('date', 'eq', '2020-12-31').T

,0,0,0,0,0,0,0,0,0,0,0
symbol,AEVA,GOEV,DNMR,VIEW,XL,RMO,EOSE,ASLE,ADN,ARRY,VIAO
date,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31,2020-12-31
period,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY,FY
revenuePerShare,0,0.000031,1.778370,None,0.154820,0.391280,0.023276,5.090317,0.000000,6.871679,6.736264
netIncomePerShare,-0.013879,-0.001089,-0.472386,None,-0.194808,-1.821889,-7.307382,0.206658,-0.088598,0.465164,-0.097115
operatingCashFlowPerShare,-0.006676,-1.298506,-0.518373,None,-0.151341,-1.302770,-2.822771,-0.297979,-0.475942,-0.962290,-0.032992
freeCashFlowPerShare,-0.006676,-1.390180,-1.956158,None,-0.152445,-1.360542,-3.205921,-0.474977,-0.475942,-0.972825,-0.161271
cashPerShare,0.000003,8.519989,14.186269,None,2.509347,12.750913,12.950904,0.714242,0.003709,0.853906,3.575606
bookValuePerShare,1.132672,6.936284,16.490263,None,2.528758,14.570099,13.083014,8.591318,0.743526,-0.637030,3.425518
tangibleBookValuePerShare,1.132672,6.936284,16.422597,None,2.520522,14.570099,13.049004,7.416444,12.346126,-2.747264,3.257774


In [242]:
stocks.search('ARRY').profile

,ARRY
price,15.9200
beta,0.0000
volAvg,4399631
mktCap,2021744512
lastDiv,0.0000
range,13.22-54.78
changes,0.0700
companyName,"Array Technologies, Inc."
currency,USD
cik,0001820721


In [244]:
ipo_details = [{
    'symbol': 'ARRY',
    'n_shares': 54.625,
    'dilutive': 7+7.125,
    'price': 22,
    'total_shares'
}]

In [247]:
df = pd.DataFrame(ipo_details)

In [251]:
df.price*df.n_shares

0    1201.75
dtype: float64

In [97]:
import pandas as pd

df = pd.read_excel('crunchrip.xlsx', header=None)
ser = pd.concat((pd.Series(['num', 'logo']), df[0]))

n_cols = 21
ser_ = ser.values.flatten().reshape(-1, n_cols)

df = pd.DataFrame(ser_[1:, 2:], columns=ser_[0, 2:])
df = df[df['Last Funding Date'] > '2018-01-01'].reset_index(drop=True)
cols = [
    'Organization Name', 'Number of Employees', 'Last Funding Type', 'Last Funding Date',
    'Last Funding Amount', 'Number of Funding Rounds', 'Total Funding Amount',
    'Funding Status', 'Top 5 Investors', 'Number of Lead Investors', 'Number of Investors',
    'Estimated Revenue Range', 'Industries', 'Headquarters Location',
    'Description', 'Most Recent Valuation Range',
    'Date of Most Recent Valuation', 'Patents Granted',
    'Most Popular Patent Class',
]

df = df[cols]

In [98]:
df.to_csv('series_b.csv')

In [102]:
investors = [inv for invs in df['Top 5 Investors'].str.split(', ').values for inv in invs]
len(set(investors))

111

In [103]:
df.shape

(31, 19)

In [104]:
df

,Organization Name,Number of Employees,Last Funding Type,Last Funding Date,Last Funding Amount,Number of Funding Rounds,Total Funding Amount,Funding Status,Top 5 Investors,Number of Lead Investors,Number of Investors,Estimated Revenue Range,Industries,Headquarters Location,Description,Most Recent Valuation Range,Date of Most Recent Valuation,Patents Granted,Most Popular Patent Class
0,Kepler Communications,11-50,Series B,2021-06-09,60000000,10,85060177,Early Stage Venture,"Creative Destruction Lab (CDL), Techstars, Can...",4,24,$1M to $10M,"Satellite Communication, Telecommunications","Toronto, Ontario, Canada",Kepler Communications is a satellite communica...,—,—,1,Electric Communication Technique
1,Liqid,11-50,Series B,2019-11-13,28000000,3,49200000,Early Stage Venture,"Panorama Point Partners, Kingston, Iron Gate C...",3,7,$10M to $50M,"Data Center, Data Center Automation, Enterpris...","Broomfield, Colorado, United States",Liqid is a information technology company that...,$100M to $500M,2019-11-14 00:00:00,74,Computing; Calculating
2,Concord,51-100,Series B,2018-12-07,—,6,41400000,Early Stage Venture,"CRV, Streamlined Ventures, Cota Capital, Alven...",3,8,$1M to $10M,"Contact Management, Data Storage, Document Man...","San Francisco, California, United States",Concord is the only contract management platfo...,—,—,0,—
3,Astrocast,51-100,Series B,2021-02-25,—,6,14077906,Early Stage Venture,"Verve Ventures, Airbus Ventures, Foundation fo...",3,6,$1M to $10M,"Electronics, Space Travel, Telecommunications","Renens, Vaud, Switzerland",Astrocast offers a bidirectional and highly se...,—,—,0,—
4,Itential,101-250,Series B,2021-04-13,20000000,2,25500000,Early Stage Venture,Elsewhere Partners,1,1,$10M to $50M,"Computer, Information Technology, Software, Te...","Atlanta, Georgia, United States",Intelligent Network Automation.,—,—,0,—
5,StorageOS,11-50,Series B,2021-02-23,10000000,4,20000000,Early Stage Venture,"Alumni Ventures Group, Bain Capital Ventures, ...",2,6,$1M to $10M,"Cloud Computing, Cloud Storage, Data Storage, ...","London, England, United Kingdom","StorageOS is a cloud native, software-defined,...",—,—,5,Computing; Calculating
6,Beewise,11-50,Series B,2020-12-15,22000000,6,38714134,Early Stage Venture,"MassChallenge, Corner Ventures, Horizon 2020, ...",4,11,$1M to $10M,"Agriculture, AgTech, Animal Feed, Food and Bev...","Bet Haemek, HaZafon, Israel",The only comprehensive solution that completel...,—,—,2,—
7,Connected2Fiber,51-100,Series B,2021-03-22,12000000,6,28160000,Early Stage Venture,"Silicon Valley Bank, Harbert Growth Partners, ...",4,6,$1M to $10M,"Analytics, Commercial Real Estate, GPS, Locati...","Milford, Massachusetts, United States",Connected2Fiber is an industry platform for co...,$10M to $50M,2019-07-30 00:00:00,—,—
8,Vesper,11-50,Series B,2018-05-15,23000000,5,38000000,Early Stage Venture,"Alumni Ventures Group, Baidu, Hyperplane Ventu...",2,13,$1M to $10M,"Consumer Electronics, Hardware, Manufacturing,...","Boston, Massachusetts, United States",Vesper is an electrical and electronic manufac...,$100M to $500M,2018-05-16 00:00:00,0,—
9,Myriota,11-50,Series B,2020-04-06,"A$28,000,000",2,32128242,Early Stage Venture,"Blue Sky Private Equity, In-Q-Tel, Singtel Inn...",3,9,$1M to $10M,"Asset Management, Internet of Things, Satellit...","Adelaide, South Australia, Australia",Myriota provides low cost IoT connectivity wit...,—,—,27,Electric Communication Technique
